In [10]:
import glob
from matplotlib import pyplot as plt
import tqdm
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import sys
import json
from sklearn.metrics import f1_score
import numpy as np
from typing import List, Tuple
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import json
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sentence_transformers import SentenceTransformer
import re
import rich
from rich import print as rprint
import os
import hashlib
from datetime import datetime
import json


/home/user/Desktop/RakutenMultimodalClassification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [13]:
CONFIG = {
    "embedder": "intfloat/multilingual-e5-large-instruct",
    "embedding_dimension": 1024,
    "use_description": True,
    "use_instruct": False,
}

EMBEDDINGS_DIMENSION = CONFIG["embedding_dimension"]
USE_DESCRIPTION = CONFIG["use_description"]
USE_INSTRUCT = CONFIG["use_instruct"]



In [14]:
class ClassificationHead(nn.Module):
    def __init__(self, input_dim, num_classes, dropout_rate=0.3):
        super().__init__()
        # Input dimension depends on whether we're using description
        combined_dim = input_dim * 2 if USE_DESCRIPTION else input_dim
        self.classifier = nn.Sequential(
            nn.Linear(combined_dim, 512),
            nn.BatchNorm1d(512),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x1, x2=None):
        if USE_DESCRIPTION:
            combined = torch.cat((x1, x2), dim=1)
        else:
            combined = x1
        return self.classifier(combined)


# load model from path
path = "models/intfloat/multilingual-e5-large-instruct-0.86f1.pt"

text_classifier = ClassificationHead(EMBEDDINGS_DIMENSION, 27)
text_classifier.load_state_dict(torch.load(path))
text_classifier.to(device)

/tmp/ipykernel_37652/2076140095.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_classifier.load_state_dict(torch.load(path))


ClassificationHead(
  (classifier): Sequential(
    (0): Linear(in_features=2048, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU(approximate='none')
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=256, out_features=27, bias=True)
  )
)

In [19]:
def get_cache_path(texts, embedder_name, use_instruct):
    """Generate a unique cache path based on input texts, embedder, and instruct setting"""
    text_hash = hashlib.md5(''.join(texts).encode()).hexdigest()
    embedder_hash = hashlib.md5(embedder_name.encode()).hexdigest()
    instruct_suffix = '_instruct' if use_instruct else ''
    return f'cache/embeddings_{embedder_hash}_{text_hash}{instruct_suffix}.pt'
# Function to get embeddings in batches

# Load model
model = SentenceTransformer(CONFIG["embedder"], trust_remote_code=True).to('cuda')
model.train()

def get_embeddings(texts, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Getting embeddings"):
        batch = texts[i:i + batch_size]
        # Only apply instruct formatting if USE_INSTRUCT is True
        processed_batch = [
            get_detailed_instruct("Match similar products based on their features, characteristics, and intended use.", text) 
            if USE_INSTRUCT and text.strip()
            else text 
            for text in batch
        ]
        with torch.no_grad():
            batch_embeddings = model.encode(processed_batch, max_length=EMBEDDINGS_DIMENSION)
            if isinstance(batch_embeddings, np.ndarray):
                batch_embeddings = torch.from_numpy(batch_embeddings)
            assert batch_embeddings.shape[1] == EMBEDDINGS_DIMENSION, f"Model output dimension mismatch. Expected {EMBEDDINGS_DIMENSION}, got {batch_embeddings.shape[1]}"
            embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)

def load_or_compute_embeddings(texts, embedder_name, batch_size=32):
    """Load embeddings from cache if they exist, otherwise compute and cache them"""
    # Create cache directory if it doesn't exist
    os.makedirs('cache', exist_ok=True)
    
    cache_path = get_cache_path(texts, embedder_name, USE_INSTRUCT)
    
    # Try to load from cache
    if os.path.exists(cache_path):
        print(f"Loading embeddings from cache: {cache_path}")
        return torch.load(cache_path)
    
    # Compute embeddings
    print("Computing new embeddings...")
    embeddings = get_embeddings(texts, batch_size)
    
    # Save to cache
    print(f"Saving embeddings to cache: {cache_path}")
    torch.save(embeddings, cache_path)
    
    return embeddings


def preprocess(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = text.lower()
    text = text.strip()
    return text


In [20]:
# Custom dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, designation_embeddings, description_embeddings, labels):
        assert designation_embeddings.shape[1] == EMBEDDINGS_DIMENSION, f"Designation embeddings dimension mismatch. Expected {EMBEDDINGS_DIMENSION}, got {designation_embeddings.shape[1]}"
        if USE_DESCRIPTION:
            assert description_embeddings.shape[1] == EMBEDDINGS_DIMENSION, f"Description embeddings dimension mismatch. Expected {EMBEDDINGS_DIMENSION}, got {description_embeddings.shape[1]}"
        self.designation_embeddings = designation_embeddings
        self.description_embeddings = description_embeddings if USE_DESCRIPTION else None
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(labels)

    def __len__(self):
        return len(self.designation_embeddings)
    
    def __getitem__(self, idx):
        if USE_DESCRIPTION:
            return (self.designation_embeddings[idx], 
                    self.description_embeddings[idx], 
                    self.labels[idx])
        return (self.designation_embeddings[idx], self.labels[idx])


df = pd.read_csv('X_test.csv')
designations = [preprocess(text) for text in df['designation'].tolist()]
descriptions = [preprocess(text) for text in df['description'].tolist()]

labels = [0] * len(designations)
for i in range(len(labels)):
    labels[i] = i

designations_embeddings = load_or_compute_embeddings(designations, CONFIG["embedder"])
descriptions_embeddings = load_or_compute_embeddings(descriptions, CONFIG["embedder"])

dataset = TextClassificationDataset(designations_embeddings, descriptions_embeddings, labels)


Computing new embeddings...


Getting embeddings: 100%|██████████| 432/432 [00:13<00:00, 32.48it/s]


Saving embeddings to cache: cache/embeddings_571f3efdd580e6d678ec91a1b96b1ca1_ff21f37c2835453e2ef54e19667bb025.pt
Computing new embeddings...


Getting embeddings: 100%|██████████| 432/432 [02:05<00:00,  3.44it/s]

Saving embeddings to cache: cache/embeddings_571f3efdd580e6d678ec91a1b96b1ca1_7247ebafd4fcd6572425f0d8704a9543.pt


In [21]:

dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

all_outputs = []

with torch.no_grad():
    for batch in dataloader:
        if USE_DESCRIPTION:
            batch_des, batch_desc, batch_labels = batch
            batch_desc = batch_desc.to(device)
        else:
            batch_des, batch_labels = batch
            batch_desc = None
        
        batch_des = batch_des.to(device)
        outputs = text_classifier(batch_des, batch_desc)
        # apply softmax
        outputs = F.softmax(outputs, dim=1)
        all_outputs.append(outputs)

all_outputs = torch.cat(all_outputs, dim=0)
print(all_outputs.shape)

# save to pt
torch.save(all_outputs, 'text_softmaxes_test.pt')

torch.Size([13812, 27])


In [22]:
# Global model and processor cache
_model = None
_processor = None


def get_model_and_processor():
    """Cache and return the model and processor"""
    global _model, _processor
    if _model is None or _processor is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        _model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
        _processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        
        # Optimize model with torch.compile() if using PyTorch 2.0+
        if hasattr(torch, 'compile'):
            _model = torch.compile(_model)
        
        _model.eval()  # Set model to evaluation mode
    return _model, _processor



def process_batch(image_paths: List[str], categories: List[str], batch_size: int = 32) -> List[List[Tuple[str, float]]]:
    """Process images in batches for better performance"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, processor = get_model_and_processor()
    
    all_similarities = []
    
    # Process images in batches
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i + batch_size]
        images = []
        valid_indices = []
        
        # Load images
        for idx, path in enumerate(batch_paths):
            try:
                image = Image.open(path)
                images.append(image)
                valid_indices.append(idx)
            except Exception as e:
                print(f"Error loading image {path}: {e}")
                all_similarities.append(None)
        
        if not images:
            continue
            
        # Process batch
        with torch.no_grad():
            inputs = processor(
                images=images,
                text=[f"a photo of {category}" for category in categories],
                return_tensors="pt",
                padding=True
            ).to(device)
            
            outputs = model(**inputs)
            similarities = outputs.logits_per_image.softmax(dim=-1)
            all_similarities.append(similarities)
    
    return all_similarities

In [25]:
import pandas as pd
df = pd.read_csv("X_test.csv")


all_paths = []

for row in df.itertuples():
    path = f"images/image_test/image_{getattr(row, 'imageid')}_product_{getattr(row, 'productid')}.jpg"
    all_paths.append(path)


print(len(all_paths))

13812


In [26]:
category_ids = {
        "10": "Single book",
        "40": "Video Game Covers",
        "50": "Game Accessories",
        "60": "Game Console",
        "1140": "Video Game Figurines",
        "1160": "Cards",
        "1180": "Movies Figurine",
        "1280": "Plush toy",
        "1281": "Tabletop Game",
        "1300": "Toy Car",
        "1301": "Game Room Accessories",
        "1302": "Outdoor Toys",
        "1320": "Baby accessories",
        "1560": "Furnitures",
        "1920": "Pillows",
        "1940": "Food & Beverages",
        "2060": "Flags and decorations",
        "2220": "Pet Supplies",
        "2280": "Journals",
        "2403": "Book collection",
        "2462": "Game Console (occasion)",
        "2522": "Office Supplies",
        "2582": "Outdoor & Garden",
        "2583": "Spa Supplies",
        "2585": "Tools & Home Improvement",
        "2705": "Literature book",
        "2905": "PC games"
    }

In [27]:
res = process_batch(all_paths, category_ids.values(), 32)
print(type(res))
print(len(res))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


<class 'list'>
432


In [28]:
full_res = torch.tensor([])

for tensor in res:
    full_res = torch.cat((full_res, tensor.to("cpu")), dim=0)

full_res = full_res.cpu().numpy()
print(full_res.shape)

(13812, 27)


In [29]:
# convert to pt
full_res = torch.tensor(full_res)

# save as pt
torch.save(full_res, "image_softmaxes_test.pt")
full_res.shape

torch.Size([13812, 27])

In [3]:
# save labels

df = pd.read_csv('train.csv')


labels = df['class']

labels



0          10
1        2280
2          50
3        1280
4        2705
         ... 
84911      40
84912    2583
84913    2280
84914    1560
84915    2522
Name: class, Length: 84916, dtype: int64

In [51]:
import torch
text_data = torch.load("text_softmaxes_test.pt")
image_data = torch.load("image_softmaxes_test.pt")

/tmp/ipykernel_37652/2110503979.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_data = torch.load("text_softmaxes_test.pt")
/tmp/ipykernel_37652/2110503979.py:3: Fu

In [45]:
for t in text_data:
    assert len(t) == 27
    assert t.sum().item() - 1 < 1e-6

for t in image_data:
    assert len(t) == 27
    assert t.sum().item() - 1 < 1e-6

assert len(text_data) == len(image_data)

image_data[0]

tensor([1.0369e-03, 3.8984e-04, 2.3128e-03, 7.8987e-04, 1.9691e-02, 5.9527e-04,
        8.2609e-02, 7.3904e-01, 1.5808e-02, 1.4739e-02, 2.3738e-03, 1.6038e-02,
        3.0135e-02, 2.4691e-04, 2.3044e-04, 4.5885e-05, 3.6524e-07, 6.7587e-02,
        8.6550e-04, 2.3893e-05, 5.1681e-04, 1.1930e-03, 1.4306e-04, 1.6683e-03,
        2.5537e-04, 2.1591e-04, 1.4530e-03])

# Model creation

In [57]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import f1_score
import numpy as np
from sklearn.preprocessing import LabelEncoder

import pandas as pd


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Custom Dataset
class CombinedDataset(Dataset):
    def __init__(self, text_data_path, image_data_path):
        self.text_data = text_data.to(device)
        self.image_data = image_data.to(device)
        assert len(self.text_data) == len(self.image_data), "Datasets must have same length"
        df = pd.read_csv("train.csv")
        # Label encode the classes to be in range [0, num_classes-1]
        self.le = LabelEncoder()
        encoded_labels = self.le.fit_transform(df["class"].values)
        # Convert encoded labels to tensor
        self.labels = torch.tensor(encoded_labels, dtype=torch.long).to(device)
        
    def __len__(self):
        return len(self.text_data)
    
    def __getitem__(self, idx):
        text_tensor = self.text_data[idx]  # Already on device
        image_tensor = self.image_data[idx]  # Already on device
        combined_input = torch.cat([text_tensor, image_tensor])
        label = self.labels[idx]
        return combined_input, label

# MLP Model
class MLP(nn.Module):
    def __init__(self, input_size=54):  # 27 + 27 = 54 input features
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 27)
        )
    
    def forward(self, x):
        return self.layers(x)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device=device):
    model = model.to(device)
    best_val_f1 = 0
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
        # Validation
        model.eval()
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)
                
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        print(val_labels, val_preds)
        
        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Loss: {train_loss/len(train_loader):.4f}')
        print(f'Validation F1 Score: {val_f1:.4f}')
        
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), 'late_merger.pt')



cuda


In [ ]:
# Main execution
def main():
    # Hyperparameters
    BATCH_SIZE = 32
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 10
    
    # Load dataset
    dataset = CombinedDataset('text_softmaxes.pt', 'image_softmaxes.pt')
    
    # Split dataset
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    
    # Initialize model, criterion, and optimizer
    model = MLP()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Train model
    train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS)

if __name__ == "__main__":
    input()
    main()

# Model inference

In [58]:
late_merger = MLP()
late_merger.load_state_dict(torch.load('models/late_merger.pt'))

dataset = CombinedDataset("text_softmaxes_test.pt", "image_softmaxes_test.pt")
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)


/tmp/ipykernel_37652/2348993138.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  late_merger.load_state_dict(torch.load('models/late_merger.pt'))


In [60]:
late_merger.eval()
print(device)
late_merger.to(device)

all_preds = []
with torch.no_grad():
    for inputs, labels  in dataloader:
        inputs, _ = inputs.to(device), labels.to(device)
        outputs = late_merger(inputs)
        best = torch.argmax(outputs, dim=1)
        all_preds.extend(best.cpu().numpy())

len(all_preds)

real_preds = dataset.le.inverse_transform(all_preds)



cuda


In [61]:
df_test = pd.read_csv('X_test.csv')
res_df = pd.DataFrame({"": df_test['Unnamed: 0'], 'prdtypecode': real_preds})
res_df.to_csv('submission.csv', index=False)